In [1]:
from biollm.task.perturbation.pert_task import PertTask


config_file = '../../config/pert/scmamba_pert.toml'
obj = PertTask(config_file)
pert_data, gene_ids = obj.make_dataset()

/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/plugins/environments/xla.py:18: DeprecationWarning: `ModuleAvailableCache` is a special case of `RequirementCache`. Please use `RequirementCache(module=...)` instead.
  from lightning.fabric.accelerators.tpu import _XLA_AVAILABLE, TPUAccelerator
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/pkg_resources/__init__.py:2348: DeprecationWarning: Deprecated call 

Using simple batchnorm instead of domain specific batchnorm


Found local copy...


before filter: (63585, 4399)
match 1.0 genes in vocab of size 4399
after filter: View of AnnData object with n_obs × n_vars = 63585 × 4399
    obs: 'condition', 'cell_type', 'dose_val', 'control', 'condition_name', 'pert_gene_in_vocab'
    var: 'gene_name', 'is_in_vocab'
    uns: 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'


Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TIMM23+ctrl' 'AMIGO3+ctrl' 'KCTD16+ctrl']
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:18
Done!
Creating dataloaders....
Done!


here1


2024-04-18 18:51:22-pert_task[line-67]-INFO: match 4399/4399 genes in vocabulary of size 60697.


In [2]:
best_model_path = obj.args.save_dir + '/best_model.pt'
best_model = obj.load_obj.load_pretrain_model(best_model_path, obj.model)

In [69]:
import torch
from torch.utils.data import DataLoader
from biollm.model.perturbation import ScgptPerturbation
from typing import Dict, Optional
import numpy as np
from anndata import AnnData
import pickle
from biollm.repo.scmamba.scmamba.mambaLM import MambaModel

def eval_scmamba_perturb(
    loader: DataLoader, model: MambaModel, device: torch.device, include_zero_gene: str
, gene_ids: np.ndarray) -> Dict:
    """
    Run model in inference mode using a given data loader
    """
    model.eval()
    model.to(device)
    pert_cat = []
    pred = []
    truth = []
    pred_de = []
    truth_de = []
    results = {}
    de_idxs = {}
    sorted_indexs = {}
    print('start to predict: ', len(loader.dataset))
    for itr, batch in enumerate(loader):
        batch.to(device)
        pert_cat.extend(batch.pert)
        for i in range(len(batch['pert'])):
            if batch["pert"][i] not in de_idxs:
                de_idxs[batch["pert"][i]] = batch.de_idx[i]
        for i in range(len(batch['pert'])):
            if batch["pert"][i] not in sorted_indexs:
                sorted_indexs[batch["pert"][i]] = batch.sorted_index[i]
        with torch.no_grad():
            p = model.pred_perturb(
                batch,
                include_zero_gene=include_zero_gene,
                gene_ids=gene_ids,
            )
            t = batch.y
            pred.extend(p.cpu())
            truth.extend(t.cpu())
            
            # Differentially expressed genes
            for itr, de_idx in enumerate(batch.de_idx):
                pred_de.append(p[itr, de_idx])
                truth_de.append(t[itr, de_idx])
    print('end')
    # all genes
    results["pert_cat"] = np.array(pert_cat)
    pred = torch.stack(pred)
    truth = torch.stack(truth)
    results["pred"] = pred.detach().cpu().numpy().astype(float)
    results["truth"] = truth.detach().cpu().numpy().astype(float)
    results['de_idx'] = de_idxs
    results['sorted_index'] = sorted_indexs
    pred_de = torch.stack(pred_de)
    truth_de = torch.stack(truth_de)
    results["pred_de"] = pred_de.detach().cpu().numpy().astype(float)
    results["truth_de"] = truth_de.detach().cpu().numpy().astype(float)
    return results

In [70]:
test_loader = pert_data.dataloader["test_loader"]
test_res = eval_scmamba_perturb(test_loader, best_model, 'cuda:2', 'all', gene_ids)

start to predict:  8695
end


In [57]:
from biollm.repo.gears.inference import evaluate, compute_metrics

test_metrics, test_pert_res = compute_metrics(test_res)

In [71]:
test_metrics

{'mse': 0.010639110014391306,
 'mse_de': 0.182111431037015,
 'pearson': 0.9865111277927033,
 'pearson_de': 0.9534139080623021}

In [ ]:
test_metrics

In [81]:
def compute_perturbation_metrics(
    results: Dict,
    ctrl_adata: AnnData,
    de_idx: Dict,
    sorted_index: Dict,
    non_zero_genes: bool = False,
    return_raw: bool = False,
    output: Optional[str] = None
) -> Dict:
    """
    Given results from a model run and the ground truth, compute metrics

    Args:
        results (:obj:`Dict`): The results from a model run
        ctrl_adata (:obj:`AnnData`): The adata of the control condtion
        non_zero_genes (:obj:`bool`, optional): Whether to only consider non-zero
            genes in the ground truth when computing metrics
        return_raw (:obj:`bool`, optional): Whether to return the raw metrics or
            the mean of the metrics. Default is False.

    Returns:
        :obj:`Dict`: The metrics computed
    """
    from scipy.stats import pearsonr

    # metrics:
    #   Pearson correlation of expression on all genes, on DE genes,
    #   Pearson correlation of expression change on all genes, on DE genes,

    metrics_across_genes = {
        "pearson": [],
        "pearson_de": [],
        "pearson_delta": [],
        "pearson_de_delta": [],
    }

    metrics_across_conditions = {
        "pearson": [],
        "pearson_delta": [],
    }

    conditions = np.unique(results["pert_cat"])
    assert not "ctrl" in conditions, "ctrl should not be in test conditions"
    condition2idx = {c: np.where(results["pert_cat"] == c)[0] for c in conditions}

    
    mean_ctrl = np.array([ctrl_adata.X[:, sorted_index[c]].mean(0) for c in conditions]).reshape(len(conditions), -1)
    assert ctrl_adata.X.max() <= 1000, "gene expression should be log transformed"

    true_perturbed = results["truth"]  # (n_cells, n_genes)
    assert true_perturbed.max() <= 1000, "gene expression should be log transformed"
    true_mean_perturbed_by_condition = np.array(
        [true_perturbed[condition2idx[c]].mean(0) for c in conditions]
    )  # (n_conditions, n_genes)
    true_mean_delta_by_condition = true_mean_perturbed_by_condition - mean_ctrl
    zero_rows = np.where(np.all(true_mean_perturbed_by_condition == 0, axis=1))[
        0
    ].tolist()
    

    pred_perturbed = results["pred"]  # (n_cells, n_genes)
    pred_mean_perturbed_by_condition = np.array(
        [pred_perturbed[condition2idx[c]].mean(0) for c in conditions]
    )  # (n_conditions, n_genes)
    pred_mean_delta_by_condition = pred_mean_perturbed_by_condition - mean_ctrl

    def corr_over_genes(x, y, conditions, res_list, skip_rows=[], non_zero_mask=None):
        """compute pearson correlation over genes for each condition"""
        for i, c in enumerate(conditions):
            if i in skip_rows:
                continue
            x_, y_ = x[i], y[i]
            if non_zero_mask is not None:
                x_ = x_[non_zero_mask[i]]
                y_ = y_[non_zero_mask[i]]
            res_list.append(pearsonr(x_, y_)[0])

    corr_over_genes(
        true_mean_perturbed_by_condition,
        pred_mean_perturbed_by_condition,
        conditions,
        metrics_across_genes["pearson"],
        zero_rows,
        non_zero_mask=true_mean_perturbed_by_condition != 0 if non_zero_genes else None,
    )
    corr_over_genes(
        true_mean_delta_by_condition,
        pred_mean_delta_by_condition,
        conditions,
        metrics_across_genes["pearson_delta"],
        zero_rows,
        non_zero_mask=true_mean_perturbed_by_condition != 0 if non_zero_genes else None,
    )
    mean_ctrl_de = np.array(
        [mean_ctrl[i, de_idx[c]] for i, c in enumerate(conditions)]
    )  # (n_conditions, n_diff_genes)
    true_mean_perturbed_by_condition_de = np.array(
        [
            true_mean_perturbed_by_condition[i, de_idx[c]]
            for i, c in enumerate(conditions)
        ]
    )  # (n_conditions, n_diff_genes)
    zero_rows_de = np.where(np.all(true_mean_perturbed_by_condition_de == 0, axis=1))[
        0
    ].tolist()
    true_mean_delta_by_condition_de = true_mean_perturbed_by_condition_de - mean_ctrl_de

    pred_mean_perturbed_by_condition_de = np.array(
        [
            pred_mean_perturbed_by_condition[i, de_idx[c]]
            for i, c in enumerate(conditions)
        ]
    )  # (n_conditions, n_diff_genes)
    pred_mean_delta_by_condition_de = pred_mean_perturbed_by_condition_de - mean_ctrl_de

    corr_over_genes(
        true_mean_perturbed_by_condition_de,
        pred_mean_perturbed_by_condition_de,
        conditions,
        metrics_across_genes["pearson_de"],
        zero_rows_de,
    )
    corr_over_genes(
        true_mean_delta_by_condition_de,
        pred_mean_delta_by_condition_de,
        conditions,
        metrics_across_genes["pearson_de_delta"],
        zero_rows_de,
    )

    if not return_raw:
        for k, v in metrics_across_genes.items():
            metrics_across_genes[k] = np.mean(v)
        for k, v in metrics_across_conditions.items():
            metrics_across_conditions[k] = np.mean(v)
    metrics = metrics_across_genes
    if output is not None:
        with open(output, 'wb') as f:
            pickle.dump(metrics, f)
    return metrics


In [82]:
de_idx = test_res['de_idx']
sorted_index = test_res['sorted_index']
test_metrics = compute_perturbation_metrics(
    test_res, pert_data.adata[pert_data.adata.obs["condition"] == "ctrl"], de_idx, sorted_index
)
print(test_metrics)

{'pearson': 0.9865111277927033, 'pearson_de': 0.9534139080623021, 'pearson_delta': 0.345619679893135, 'pearson_de_delta': 0.7731520751238459}


In [ ]:
{'pearson': 0.9865111277927033, 'pearson_de': 0.9534139080623021, 'pearson_delta': 0.9929274519015182, 'pearson_de_delta': 0.9556105969532533}

In [53]:
from biollm.repo.gears.inference import deeper_analysis, non_dropout_analysis
import numpy as np

deeper_res = deeper_analysis(pert_data.adata, test_res)
non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)

metrics = ["pearson_delta", "pearson_delta_de"]
metrics_non_dropout = [
    "pearson_delta_top20_de_non_dropout",
    "pearson_top20_de_non_dropout",
]
subgroup_analysis = {}
for name in pert_data.subgroup["test_subgroup"].keys():
    subgroup_analysis[name] = {}
    for m in metrics:
        subgroup_analysis[name][m] = []

    for m in metrics_non_dropout:
        subgroup_analysis[name][m] = []

for name, pert_list in pert_data.subgroup["test_subgroup"].items():
    for pert in pert_list:
        for m in metrics:
            subgroup_analysis[name][m].append(deeper_res[pert][m])

        for m in metrics_non_dropout:
            subgroup_analysis[name][m].append(non_dropout_res[pert][m])

for name, result in subgroup_analysis.items():
    for m in result.keys():
        mean_value = np.mean(subgroup_analysis[name][m])
        print("test_" + name + "_" + m + ": " + str(mean_value))


test_combo_seen0_pearson_delta: nan
test_combo_seen0_pearson_delta_de: nan
test_combo_seen0_pearson_delta_top20_de_non_dropout: nan
test_combo_seen0_pearson_top20_de_non_dropout: nan
test_combo_seen1_pearson_delta: nan
test_combo_seen1_pearson_delta_de: nan
test_combo_seen1_pearson_delta_top20_de_non_dropout: nan
test_combo_seen1_pearson_top20_de_non_dropout: nan
test_combo_seen2_pearson_delta: nan
test_combo_seen2_pearson_delta_de: nan
test_combo_seen2_pearson_delta_top20_de_non_dropout: nan
test_combo_seen2_pearson_top20_de_non_dropout: nan
test_unseen_single_pearson_delta: 0.3456196798931348
test_unseen_single_pearson_delta_de: 0.773152075123846
test_unseen_single_pearson_delta_top20_de_non_dropout: 0.7728769223680013
test_unseen_single_pearson_top20_de_non_dropout: 0.9508552704076959
